---
# Feature Engineering

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import ast
import numpy as np 
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
archivo1="../DataSets/steam_games.parquet"
df= pd.read_parquet(archivo1)

In [3]:
archivo2="../DataSets/user_review.parquet"
df2= pd.read_parquet(archivo2)

In [4]:
archivo3="../DataSets/user_items.parquet"
df3= pd.read_parquet(archivo3)

## Iniciamos con el analisis de sentimientos

In [6]:
#Cargamos unas funciones para ayudarnos
def analizar_sentimiento(review):
    """
    Analiza el sentimiento de una reseña.

    Args:
        review (str): Texto de la reseña.

    Returns:
        int: Código de sentimiento (0: Negativo, 1: Neutral, 2: Positivo).
    """
    if not review:
        return 1  # Neutral

    analyzer = SentimentIntensityAnalyzer()
    compound_score = analyzer.polarity_scores(review)['compound']

    if compound_score > 0.1:
        return 2  # Positivo
    elif compound_score < -0.1:
        return 0  # Negativo
    else:
        return 1  # Neutral

In [ ]:
df2 ["sentiment_analisis"]= df2["review"].astype(str).apply(analizar_sentimiento)
df2.head(5)

Buscamos una reseña para testear nuestro analisis

In [111]:
df2["review"][15]

'It was a great game from what I played, right now I need to find the actual download.'

In [112]:
df2["sentiment_analisis"][15]

2

Buscamos otra para enriquecer las pruebas 

In [113]:
df2["review"][90]

"I just can't stop playing it that good <:"

In [114]:
df2["sentiment_analisis"][90]

2

In [115]:
df2.drop(columns="review", inplace=True)

In [116]:
df2.to_parquet("../DataSets/Sentiment_Analisis.parquet")

## Desarrollo de la API, funciones y endpoints

Empezamos a crear nuestros datasets para la API

In [5]:
games=pd.read_parquet("../DataSets/steam_games.parquet")
sentiment_analisis=pd.read_parquet("../DatosML/sentiment_analisis_limpio.parquet")
user_item=pd.read_parquet("../DataSets/user_items.parquet")

In [6]:
games.head(2)

,publisher,genres,title,release_date,tags,specs,price,early_access,id,developer
0,Kotoshiro,Action,Lost Summoner Kitty,2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,False,761140,Kotoshiro
1,Kotoshiro,Casual,Lost Summoner Kitty,2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,False,761140,Kotoshiro


In [7]:
sentiment_analisis.head(2)

,user_id,posted,item_id,helpful,recommend,sentiment_analisis
0,76561197970982479,2011,1250,No ratings yet,True,2
1,76561197970982479,2011,22200,No ratings yet,True,2


In [8]:
user_item.head(2)

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479


In [9]:
#Se extraen las columnas para generar un nuevo dataframe y correr la función
genre= games[["genres","id"]]
#Cambiamos "id" a "item_id" para poder utilizar el merge
genre= genre.rename(columns={"id":"item_id"})
genre

,genres,item_id
0,Action,761140
1,Casual,761140
2,Indie,761140
3,Simulation,761140
4,Strategy,761140
...,...,...
67479,Indie,610660
67480,Racing,610660
67481,Simulation,610660
67482,Casual,658870


In [10]:
play_time= user_item[["user_id","item_id","playtime_forever"]]
play_time

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0
...,...,...,...
5094100,76561198329548331,346330,0
5094101,76561198329548331,373330,0
5094102,76561198329548331,388490,3
5094103,76561198329548331,521570,4


In [11]:
developer_opinion=pd.read_parquet("../Funciones/developer_opinion.parquet")

In [15]:
playtime = pd.merge(developer_opinion[['item_id','sentiment_analisis','user_id','recommend', 'title']], user_item[["item_name",'playtime_forever', 'user_id']], left_on='user_id', right_on='user_id')
playtime

,item_id,sentiment_analisis,user_id,recommend,title,item_name,playtime_forever
0,1250,2,76561197970982479,1,Killing Floor,Counter-Strike,6
1,1250,2,76561197970982479,1,Killing Floor,Team Fortress Classic,0
2,1250,2,76561197970982479,1,Killing Floor,Day of Defeat,7
3,1250,2,76561197970982479,1,Killing Floor,Deathmatch Classic,0
4,1250,2,76561197970982479,1,Killing Floor,Half-Life: Opposing Force,0
...,...,...,...,...,...,...,...
15656410,209120,2,pigeonie,1,Street Fighter X Tekken,Hatoful Boyfriend,319
15656411,209120,2,pigeonie,1,Street Fighter X Tekken,The Expendabros,6
15656412,209120,2,pigeonie,1,Street Fighter X Tekken,Five Nights at Freddy's,0
15656413,209120,2,pigeonie,1,Street Fighter X Tekken,Fuse,4


In [16]:
#Exportamos nuestro archivo a parquet
playtime.to_parquet("../Funciones/playtime.parquet")

Armamos nuestros archivos para las funciones, con el objetivo de mejorar el rendimiento de nuestra api

In [17]:
genre= play_time.merge(genre, on="item_id")
genre.to_parquet("../Funciones/genre_play.parquet")

Armamos nuestro otro archivo

In [18]:
prueba1 = pd.merge(sentiment_analisis[['item_id','sentiment_analisis','user_id','recommend']], games[["title",'release_date','id','developer']], left_on='item_id', right_on='id')

prueba1['recommend'] = prueba1['recommend'].replace({True: 1, False: 0}) #cambio los valores de la columna recommend, para poder utilizarlos de una forma mas sencilla en mi funcion

for index, row in prueba1.iterrows(): #en este iterador creo la nueva columna y le coloco 1 si recommend + opinion es 2
    if row[1] == 2 and row[3] == 1:
        prueba1.at[index, 'Positivo'] = 1
    else:
        prueba1.at[index, 'Positivo'] = 0

prueba1['Positivo'] = prueba1['Positivo'].astype(int) #Convierto la columna 'Positivo' a números enteros

prueba1 

,item_id,sentiment_analisis,user_id,recommend,title,release_date,id,developer,Positivo
0,1250,2,76561197970982479,1,Killing Floor,2009,1250,Tripwire Interactive,1
1,1250,2,death-hunter,1,Killing Floor,2009,1250,Tripwire Interactive,1
2,1250,0,DJKamBer,1,Killing Floor,2009,1250,Tripwire Interactive,0
3,1250,1,diego9031,1,Killing Floor,2009,1250,Tripwire Interactive,0
4,1250,1,76561198081962345,1,Killing Floor,2009,1250,Tripwire Interactive,0
...,...,...,...,...,...,...,...,...,...
124004,262850,1,wayfeng,1,The Journey Down: Chapter Two,2014,262850,SkyGoblin,0
124005,431510,2,76561198254993106,1,Mystic Destinies: Serendipity of Aeons,2016,431510,Aeon Dream Studios,1
124006,431510,2,76561198254993106,1,Mystic Destinies: Serendipity of Aeons,2016,431510,Aeon Dream Studios,1
124007,431510,2,76561198254993106,1,Mystic Destinies: Serendipity of Aeons,2016,431510,Aeon Dream Studios,1


In [184]:
prueba1.to_parquet("../Funciones/developer_opinion.parquet")

Prueba función punto 1

In [61]:
df_games=pd.read_parquet("../Funciones/genre_play.parquet")
developer_opinion=pd.read_parquet("../Funciones/developer_opinion.parquet")
playtime=pd.read_parquet("../Funciones/playtime.parquet")

In [69]:
from fastapi import FastAPI, HTTPException
from typing import Dict

app = FastAPI()

# Validar género
def validate_genre(genre):
    genre_name = genre.capitalize()
    df_filtered = [games['genres'].str.contains(genre, case=False, na=False)]
    if df_filtered.empty:
        raise HTTPException(status_code=404, detail=f"Género {genre_name} no encontrado")

# Obtener año con más horas jugadas para un género dado
@app.get('/PlayTimeGenre', response_model=Dict[str, int])
def play_time_genre(genre: str):
    """
    Devuelve el año de lanzamiento con más horas jugadas para un género dado.
    """
    validate_genre(genre)

    genre_name = genre.capitalize()
    df_filtered = playtime[playtime['genres'].str.contains(genre, case=False, na=False)]

    # Agrupar por año y sumar las horas jugadas
    playtime_by_year = df_filtered.groupby('release_date')['playtime_forever'].sum().reset_index()

    # Obtener el año con más horas jugadas
    most_played_year = playtime_by_year.loc[playtime_by_year['playtime_forever'].idxmax()]

    result = {f"Año de lanzamiento con más horas jugadas para Género {genre_name}": most_played_year['release_date']}
    return result


In [ ]:
play_time_genre("Action")

Prueba función del punto 2

In [65]:
# Primera función
def validate_genre(genre):
    genre_name = genre.capitalize()
    df_filtered = games[games['genres'].str.contains(genre, case=False, na=False)]
    if df_filtered.empty:
        raise HTTPException(status_code=404, detail=f"Género {genre_name} no encontrado")

@app.get('/UserForGenre')
def user_for_genre(genre: str):
    """
    Obtiene el usuario con más horas jugadas para un género dado.
    """
    validate_genre(genre)

    genre_name = genre.capitalize()
    df_filtered = games[games['genres'].str.contains(genre, case=False, na=False)]

    df_merged = pd.merge(playtime, df_filtered[['id', 'release_date']], left_on='item_id', right_on='id')
    user_with_most_playtime = df_merged.groupby('user_id')['playtime_forever'].sum().idxmax()
    playtime_by_year = df_merged.groupby(['release_date', 'user_id'])['playtime_forever'].sum().reset_index()
    playtime_by_year = playtime_by_year[playtime_by_year['user_id'] == user_with_most_playtime]
    playtime_by_year = playtime_by_year.rename(columns={'release_date': 'Año', 'playtime_forever': 'Horas'})

    result = {
        f"Usuario con más horas jugadas para Género {genre_name}": user_with_most_playtime,
        "Horas jugadas": [{"Año": str(row['Año']), "Horas": row['Horas']} for _, row in playtime_by_year.iterrows()]
    }
    return result


In [66]:
validate_genre("Action")
user_for_genre("Action")

{'Usuario con más horas jugadas para Género Action': 'Terminally-Chill',
 'Horas jugadas': [{'Año': '2007', 'Horas': 3468014},
  {'Año': '2012', 'Horas': 6936028},
  {'Año': '2013', 'Horas': 6936028},
  {'Año': '2014', 'Horas': 5202021},
  {'Año': '2015', 'Horas': 3468014}]}

Prueba función punto 3

In [ ]:
# Segunda función
# Obtener top 3 de juegos MÁS recomendados
from typing import List


# Validar año
def validate_year(year):
    if year not in developer_opinion['release_date'].unique():
        raise HTTPException(status_code=404, detail=f"Año {year} no encontrado")

# Obtener top 3 de juegos MÁS recomendados
@app.get('/UsersRecommend', response_model=List[dict])
def users_recommend(year: int):
    """
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    """
    validate_year(year)

    # Filtrar reseñas para el año y con recomendación positiva
    df_filtered = developer_opinion[(developer_opinion['release_date'] == year) & (developer_opinion['recommend'] == True)]

    # Ordenar por puntuación positiva en orden descendente y tomar las primeras 3
    df_sorted = df_filtered.sort_values(by='Positivo', ascending=False).head(3)

    # Crear el resultado con el formato especificado
    result = [{"Puesto {}".format(i+1): {"Título": title, "Puntuación Positiva": positive_score}} for i, (title, positive_score) in enumerate(zip(df_sorted['title'], df_sorted['Positivo']))]
    
    return result





In [38]:
users_recommend(year=2004)

[{'Puesto 1': {'Título': 'Half-Life 2', 'Puntuación Positiva': 1}},
 {'Puesto 2': {'Título': 'Counter-Strike: Source', 'Puntuación Positiva': 1}},
 {'Puesto 3': {'Título': 'Counter-Strike: Source', 'Puntuación Positiva': 1}}]

Prueba función del punto 4

In [48]:
developer_opinion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124009 entries, 0 to 124008
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   item_id             124009 non-null  int64 
 1   sentiment_analisis  124009 non-null  int64 
 2   user_id             124009 non-null  object
 3   recommend           124009 non-null  int64 
 4   title               124009 non-null  object
 5   release_date        124009 non-null  int64 
 6   id                  124009 non-null  int64 
 7   developer           124009 non-null  object
 8   Positivo            124009 non-null  int32 
dtypes: int32(1), int64(5), object(3)
memory usage: 9.0+ MB


In [39]:
from fastapi import FastAPI, HTTPException
from typing import List

app = FastAPI()



# Validar año
def validate_year(year):
    if year not in developer_opinion['release_date'].unique():
        raise HTTPException(status_code=404, detail=f"Año {year} no encontrado")

# Obtener top 3 de desarrolladoras con juegos MENOS recomendados
@app.get('/UsersWorstDeveloper', response_model=List[dict])
def users_worst_developer(year: int):
    """
    Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado.
    """
    validate_year(year)

    # Filtrar reseñas para el año y con recomendación negativa
    df_filtered = developer_opinion[(developer_opinion['release_date'] == year) & (developer_opinion['sentiment_analisis'].isin([0, 1, 2]))]

    # Agrupar por desarrolladora y contar las reseñas por categoría
    df_grouped = df_filtered.groupby(['developer', 'sentiment_analisis'])['sentiment_analisis'].count().unstack(fill_value=0).reset_index()

    # Renombrar las columnas
    df_grouped.columns = ['developer', 'Neutral', 'Negativo', 'Positivo']

    # Ordenar por puntos negativos en orden descendente y tomar las primeras 3
    df_sorted = df_grouped.sort_values(by='Negativo', ascending=False).head(3)

    # Crear el resultado con el formato especificado
    result = [{"Puesto {}".format(i+1): {"Desarrolladora": developer, "Puntuación Negativa": negative_points}} for i, (developer, negative_points) in enumerate(zip(df_sorted['developer'], df_sorted['Negativo']))]
    
    return result



In [30]:
validate_year(year=2012)
users_worst_developer(year=2012)

[{'Puesto 1': {'Desarrolladora': 'Valve', 'Puntuación Negativa': 1005}},
 {'Puesto 2': {'Desarrolladora': 'Wild Shadow Studios',
   'Puntuación Negativa': 275}},
 {'Puesto 3': {'Desarrolladora': 'Gearbox Software,Aspyr (Mac &amp; Linux)',
   'Puntuación Negativa': 222}}]

Prueba función punto 4

In [59]:
@app.get('/sentiment_analysis')
def sentiment_analysis(developer: str):
    """
    Devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros
    de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
    """
    # Convertir la columna 'developer' a str y filtrar valores nulos o extraños
    df_filtered = developer_opinion[developer_opinion['developer'].notnull()]
    df_filtered['developer'] = df_filtered['developer'].astype(str)

    # Filtrar por desarrolladora
    df_filtered = df_filtered[df_filtered['developer'] == developer]

    if df_filtered.empty:
        raise HTTPException(status_code=404, detail=f"Desarrolladora '{developer}' no encontrada")

    # Realizar el análisis de sentimiento
    sentiment_counts = df_filtered['sentiment_analisis'].value_counts().to_dict()

    # Ajustar el resultado para reflejar que 2 es positivo, 0 es neutral y 1 es negativo
    adjusted_sentiments = {2: "Positivo", 0: "Neutral", 1: "Negativo"}
    sentiment_counts = {adjusted_sentiments[key]: value for key, value in sentiment_counts.items()}

    result = {developer: sentiment_counts}
    return result


In [60]:
sentiment_analysis('Ubisoft')

{'Ubisoft': {'Positivo': 126, 'Neutral': 64, 'Negativo': 38}}

## Modelo de recomendación

In [26]:
def encontrar_juegos_similares(id_juego, modelo_final):
    """
    Encuentra juegos similares a un juego dado por su ID.

    Parameters:
    - id_juego: ID del juego para el cual se desean encontrar juegos similares.
    - modelo_render: DataFrame que contiene las características de los juegos.

    Returns:
    - Lista de nombres de juegos similares.
    """
    # Encuentra el índice del juego ingresado por ID
    juego_indice = modelo_final.index[modelo_final['id'] == id_juego].tolist()

    # Verifica si el juego con el ID especificado existe en la base de datos
    if not juego_indice:
        return f"El juego con el ID {id_juego} no existe en la base de datos."

    juego_indice = juego_indice[0]

    # Extrae las características del juego ingresado
    juego_caracteristicas = modelo_final.iloc[juego_indice, 3:].values.reshape(1, -1)

    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_render = cosine_similarity(modelo_final.iloc[:, 3:], juego_caracteristicas)

    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6].flatten()[1:]

    # Obtiene los juegos más similares en función de los índices
    juegos_similares = modelo_final.iloc[indices_juegos_similares]['title'].tolist()

    return juegos_similares


---
